# Computing on X: Highly Variable Genes

*Goal:* demonstrate larger-than-core computation on X.

This demo finds highly variable genes in a user-specified cell selection. It is similar to the [scanpy.pp.high_variable_genes](https://scanpy.readthedocs.io/en/stable/generated/scanpy.pp.highly_variable_genes.html) function, when called with `flavor='seurat_v3'`.

*NOTE*: when query results are small, it may be easier to use the SOMAExperment Query class to extract an AnnData, and then just compute over that. This notebook is showing means of incrementally processing larger-than-core (RAM) data, where incremental (online) algorithms are used.

In [1]:
import cell_census
import tiledbsoma as soma

census = cell_census.open_soma()
human = census["census_data"]["homo_sapiens"]

In [2]:
import numpy as np
import pandas as pd

from cell_census.compute import OnlineMatrixMeanVariance


def highly_variable_genes(query: soma.experiment_query.ExperimentAxisQuery, n_top_genes: int = 10) -> pd.DataFrame:
    """
    Acknowledgements: scanpy highly variable genes implementation, github.com/scverse/scanpy
    """

    try:
        import skmisc.loess
    except ImportError:
        raise ImportError("Please install skmisc package via `pip install --user scikit-misc")

    var_df = (
        query.var(column_names=["soma_joinid", "feature_id", "feature_name"])
        .concat()
        .to_pandas()
        .set_index("soma_joinid")
    )

    indexer = query.get_indexer()
    mvn = OnlineMatrixMeanVariance(query.n_obs, query.n_vars)
    for arrow_tbl in query.X("raw").tables():
        var_dim = indexer.var_index(arrow_tbl["soma_dim_1"])
        data = arrow_tbl["soma_data"].to_numpy()
        mvn.update(var_dim, data)

    u, v = mvn.finalize()
    var_df = var_df.assign(means=pd.Series(u, index=var_df.index), variances=pd.Series(v, index=var_df.index))

    estimated_variances = np.zeros((len(var_df),), dtype=np.float64)
    not_const = v > 0
    y = np.log10(v[not_const])
    x = np.log10(u[not_const])
    model = skmisc.loess.loess(x, y, span=0.3, degree=2)
    model.fit()
    estimated_variances[not_const] = model.outputs.fitted_values
    reg_std = np.sqrt(10**estimated_variances)

    # A second pass over the data is required because the clip value
    # is determined by the first pass
    N = query.n_obs
    vmax = np.sqrt(N)
    clip_val = reg_std * vmax + u
    counts_sum = np.zeros((query.n_vars,), dtype=np.float64)  # clipped
    squared_counts_sum = np.zeros((query.n_vars,), dtype=np.float64)  # clipped
    for arrow_tbl in query.X("raw").tables():
        var_dim = indexer.var_index(arrow_tbl["soma_dim_1"])
        data = arrow_tbl["soma_data"].to_numpy()
        # clip
        mask = data > clip_val[var_dim]
        data = data.copy()
        data[mask] = clip_val[var_dim[mask]]
        np.add.at(counts_sum, var_dim, data)
        np.add.at(squared_counts_sum, var_dim, data**2)

    norm_gene_vars = (1 / ((N - 1) * np.square(reg_std))) * (
        (N * np.square(u)) + squared_counts_sum - 2 * counts_sum * u
    )
    norm_gene_vars = norm_gene_vars.reshape(1, -1)

    # argsort twice gives ranks, small rank means most variable
    ranked_norm_gene_vars = np.argsort(np.argsort(-norm_gene_vars, axis=1), axis=1)

    # this is done in SelectIntegrationFeatures() in Seurat v3
    ranked_norm_gene_vars = ranked_norm_gene_vars.astype(np.float32)
    num_batches_high_var = np.sum((ranked_norm_gene_vars < n_top_genes).astype(int), axis=0)
    ranked_norm_gene_vars[ranked_norm_gene_vars >= n_top_genes] = np.nan
    ma_ranked = np.ma.masked_invalid(ranked_norm_gene_vars)
    median_ranked = np.ma.median(ma_ranked, axis=0).filled(np.nan)

    var_df = var_df.assign(
        highly_variable_nbatches=pd.Series(num_batches_high_var, index=var_df.index),
        highly_variable_rank=pd.Series(median_ranked, index=var_df.index),
        variances_norm=pd.Series(np.mean(norm_gene_vars, axis=0), index=var_df.index),
    )

    sorted_index = (
        var_df[["highly_variable_rank", "highly_variable_nbatches"]]
        .sort_values(
            ["highly_variable_rank", "highly_variable_nbatches"],
            ascending=[True, False],
            na_position="last",
        )
        .index
    )
    var_df["highly_variable"] = False
    var_df = var_df.drop(columns=["highly_variable_nbatches"])
    var_df.loc[sorted_index[: int(n_top_genes)], "highly_variable"] = True
    return var_df

To use this funtion, which is also available in `cell_census.compute`, open a ExperimentQuery, and pass to the function as a parameter.

In [3]:
with human.axis_query(
    measurement_name="RNA",
    obs_query=soma.experiment_query.AxisQuery(value_filter="tissue == 'brain'"),
) as query:
    hvg = highly_variable_genes(query)

display(hvg[hvg.highly_variable])

,feature_id,feature_name,means,variances,highly_variable_rank,variances_norm,highly_variable
soma_joinid,,,,,,,
14173,ENSG00000011465,DCN,1.469808,2.799014e+04,9.0,19.943684,True
17553,ENSG00000234745,HLA-B,2.593248,6.411781e+04,5.0,20.669148,True
17949,ENSG00000185885,IFITM1,5.726879,4.180693e+05,0.0,26.027483,True
18020,ENSG00000163453,IGFBP7,9.272079,1.508529e+06,1.0,25.324218,True
22441,ENSG00000111341,MGP,3.334319,2.661685e+05,7.0,20.188258,True
28680,ENSG00000112562,SMOC2,0.545531,5.397146e+03,8.0,20.102508,True
29507,ENSG00000105467,SYNGR4,0.883362,3.483103e+04,6.0,20.313552,True
40818,ENSG00000253504,MTCYBP19,5.558539,3.219080e+05,2.0,24.388457,True
42700,ENSG00000270225,MTCO2P22,8.855432,9.335231e+05,3.0,23.784307,True
